In [19]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

df = pd.read_csv("edu_enrollees.csv")
df.head(2)

,enrollee_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target,Xgrp
0,8949.0,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36.0,1.0,train
1,29725.0,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47.0,0.0,train


In [47]:
df = df.drop(columns = ["city", "company_size", "company_type"])
# df.dtypes
df.isna().sum()
df = df.dropna()
df.isna().sum()

df["experience"].unique(), df["last_new_job"].unique()
df = df.loc[~df["experience"].isin([">20", "<1"]), ]
df = df.loc[~df["last_new_job"].isin([">4", "never"]), ]
df["experience"] = df["experience"].astype("int")
df["last_new_job"] = df["last_new_job"].astype("int")
df_base = df.reset_index(drop = True)

# Q1.

In [48]:
df_q1 = df_base[["relevant_experience", "target"]].copy()
df_A = df_q1.loc[df_q1["relevant_experience"] == "No relevant experience", ]
df_B = df_q1.loc[df_q1["relevant_experience"] == "Has relevant experience", ]

df_A["target"].value_counts(normalize=True) # 0.0 : 0.617, 1.0: 0.3828
df_B["target"].value_counts(normalize=True) # 0.0 : 0.7840, 1.0 : 0.2159

stat_A = df_A["target"].value_counts(normalize=True)[1]
stat_B = df_B["target"].value_counts(normalize=True)[1]
round(stat_A / stat_B, 2) #1.77

# Trick 으로 아래처럼 해도 된다.
df_q1.groupby("relevant_experience")["target"].mean()

"""
enrollee_id	city	city_development_index	gender	relevant_experience	enrolled_university	education_level	major_discipline	experience	company_size	company_type	last_new_job	training_hours	target	Xgrp
0	8949.0	city_103	0.920	Male	Has relevant experience	no_enrollment	Graduate	STEM	>20	NaN	NaN	1	36.0	1.0	train
1	29725.0	city_40	0.776	Male	No relevant experience	no_enrollment	Graduate	STEM	15	50-99	Pvt Ltd	>4	47.0	0.0	train
relevant_experience
Has relevant experience    0.215911
No relevant experience     0.382873
Name: target, dtype: float64
"""


'\nenrollee_id\tcity\tcity_development_index\tgender\trelevant_experience\tenrolled_university\teducation_level\tmajor_discipline\texperience\tcompany_size\tcompany_type\tlast_new_job\ttraining_hours\ttarget\tXgrp\n0\t8949.0\tcity_103\t0.920\tMale\tHas relevant experience\tno_enrollment\tGraduate\tSTEM\t>20\tNaN\tNaN\t1\t36.0\t1.0\ttrain\n1\t29725.0\tcity_40\t0.776\tMale\tNo relevant experience\tno_enrollment\tGraduate\tSTEM\t15\t50-99\tPvt Ltd\t>4\t47.0\t0.0\ttrain\nrelevant_experience\nHas relevant experience    0.215911\nNo relevant experience     0.382873\nName: target, dtype: float64\n'

# Q2.

In [49]:
df_q2_obj = df_base.iloc[:, 2:7].copy()
df_q2_obj.head()

,gender,relevant_experience,enrolled_university,education_level,major_discipline
0,Male,Has relevant experience,no_enrollment,Graduate,STEM
1,Male,Has relevant experience,no_enrollment,Graduate,STEM
2,Female,Has relevant experience,no_enrollment,Graduate,Arts
3,Male,Has relevant experience,no_enrollment,Masters,STEM
4,Male,Has relevant experience,no_enrollment,Masters,STEM


In [50]:
df_q2_dum = pd.get_dummies(df_q2_obj, dtype="int")
# 시험볼때는 아래처럼
df_q2_dum = pd.get_dummies(df_q2_obj, columns=df_q2_obj.columns, dtype="int")

df_q2_dum.columns.to_series().reset_index() # 와 콜롬명 언제 다 쓰냐.. 이렇게 하자.
df_q2_dum = df_q2_dum.drop(columns = df_q2_dum.columns[[2,4,7,10,16]])
df_q2_dum

cols = ["city_development_index", "experience", "last_new_job", "training_hours"]
df_job2 = pd.concat([df_base[["target", "Xgrp"] + cols], df_q2_dum], axis = 1)
df_job2.head()

#odds ratio
#상수항을 제외하라 하면 LogisticRegression 의 fit_intercept 를 False로 하면 된다.
model_lr = LogisticRegression(C=100000, max_iter=1000, solver="liblinear", random_state=123)
model_lr.fit(X=df_job2.drop(columns=["target", "Xgrp"]), y=df_job2["target"])
model_lr.coef_ # 회귀 계수

pd.DataFrame(np.exp(model_lr.coef_), columns = df_job2.columns[2:]).T  # T는 그냥 보기 편하게

np.exp(model_lr.coef_).max() # 1.6738948061770285
np.floor(np.exp(model_lr.coef_).max())

1.0

# Q3.

In [51]:
df_train = df_job2.loc[df_job2["Xgrp"] == "train", ]
df_test = df_job2.loc[df_job2["Xgrp"] == "test", ]

model_knn = KNeighborsClassifier(n_neighbors=5, p=2) # 맨하탄 쓰고 싶으면 p를 1로 바꾸면 된다.
model_knn.fit(X=df_train.drop(columns=["target", "Xgrp"]), y = df_train["target"])
pred = model_knn.predict(df_test.drop(columns = ["target", "Xgrp"]))
y_t = df_test["target"]
y_p = pred

# accuracy 구해보자
df_tab = pd.crosstab(y_t, y_p)
df_tab.iloc[0,1] #이것도 가능한데.. values 로 어레이로 가도 된다.
df_tab.values.diagonal().sum() / df_tab.values.sum() # 이건 그냥 추가로 알기만 하면 된다. accuracy 계산하는 방법이다 ㅋㅋ

# 라이브러리로 하자.
from sklearn.metrics import accuracy_score
round(accuracy_score(y_true = y_t, y_pred=y_p), 2)


AttributeError: 'Flags' object has no attribute 'c_contiguous'